# 如何从工具中访问 RunnableConfig

```{=mdx}
:::info 预备知识

本指南假定您已熟悉以下概念：

- [LangChain 工具](/docs/concepts/tools)
- [自定义工具](/docs/how_to/custom_tools)
- [LangChain 表达式语言 (LCEL)](/docs/concepts/lcel)

:::
```

工具是可运行对象（runnable），在接口层面，您可以像对待其他任何可运行对象一样对待它们——可以正常调用 `invoke()`、`batch()` 和 `stream()` 方法。但是，在编写自定义工具时，您可能需要调用其他可运行对象，例如聊天模型或检索器。为了正确追踪和配置这些子调用，您需要手动访问并传入工具当前的 [`RunnableConfig`](https://api.js.langchain.com/interfaces/langchain_core.runnables.RunnableConfig.html) 对象。

本指南介绍了如何为通过不同方式创建的自定义工具执行此操作。

## 从 `tool` 方法中访问

对于使用 [`tool`](https://api.js.langchain.com/functions/langchain_core.tools.tool-1.html) 辅助方法创建的自定义工具，访问 `RunnableConfig` 对象非常简单——它始终是传递给您的自定义函数的第二个参数。以下是一个示例：

In [1]:
import { z } from "zod";
import { tool } from "@langchain/core/tools";
import type { RunnableConfig } from "@langchain/core/runnables";

const reverseTool = tool(
  async (input: { text: string }, config?: RunnableConfig) => {
    const originalString = input.text + (config?.configurable?.additional_field ?? "");
    return originalString.split("").reverse().join("");
  }, {
    name: "reverse",
    description: "A test tool that combines input text with a configurable parameter.",
    schema: z.object({
      text: z.string()
    }),
  }
);

然后，如果我们使用包含 `configurable` 字段的 `config` 调用该工具，则可以看到 `additional_field` 被正确传递：

In [2]:
await reverseTool.invoke(
  {text: "abc"}, {configurable: {additional_field: "123"}}
)

321cba


## 下一步
您现在已经了解了如何在工具内部配置和流式传输事件。接下来，请查看以下指南以了解更多关于使用工具的内容：
- 将[工具结果传递回模型](/docs/how_to/tool_results_pass_to_model)
- 构建[使用工具的链和代理](/docs/how_to#tools)
- 从模型获取[结构化输出](/docs/how_to/structured_output/)